# Imports

In [10]:
import pandas as pd
import pymc3 as pm
import numpy as np
import os
import sys
import json
import theano.tensor as tt
import arviz as az
from models import abstraction_simulation_pp
sys.path.insert(0, os.path.abspath('../python'))


pygame 2.0.0.dev6 (SDL 2.0.10, python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Load data

## Empirical data

In [3]:
datadir = "../experiments/experiment3/data/cleaned_data.json"
data = pd.read_json(datadir)

In [4]:
data.head()

,trial_index,subject_id,exp_condition,rt,response_,task,correct_response,correct,task_condition,scene,...,zlogrt,trial_condition,response,participant_accuracy,trial_accuracy,scene_accuracy,sim_time,collision,straight_path,3
0,0,61716480d6da1714ff60ed0d,j,884,f,response,f,True,No,low_nocol_yessp_1,...,-0.681529,No,No,0.895833,0.765957,0.936170,low,nocol,yessp,1.mp4
1,1,61716480d6da1714ff60ed0d,j,726,f,response,j,False,Yes,low_yescol_yessp_2,...,-1.182728,Yes,No,0.895833,0.702128,0.872340,low,yescol,yessp,2.mp4
2,2,61716480d6da1714ff60ed0d,j,885,f,response,f,True,No,low_nocol_nosp_2,...,-0.678651,No,No,0.895833,0.914894,0.893617,low,nocol,nosp,2.mp4
3,3,61716480d6da1714ff60ed0d,j,981,f,response,f,True,No,med_nocol_nosp_3,...,-0.416518,No,No,0.895833,0.787234,0.914894,med,nocol,nosp,3.mp4
4,4,61716480d6da1714ff60ed0d,j,1189,j,response,j,True,Yes,high_yescol_yessp_2,...,0.072943,Yes,Yes,0.895833,0.893617,1.000000,high,yescol,yessp,2.mp4


## Scene arguments

In [5]:
scenedir = "../data/json/pilot3/"
scene_files = [scene_json for scene_json in os.listdir(scenedir) if scene_json.endswith('.json')]

In [6]:
# Used in my_model
scene_args = {}
for file in scene_files:
    with open(scenedir+file, 'r') as f:
        sargs = (json.loads(f.read()))
        scene_args[sargs['name'].split(".")[0]] = sargs

# Model

In [8]:
modeldir = "../data/model/pilot3/abstraction_model_pp_fit.json"
model_fit = pd.read_json(modeldir)

In [9]:
model_fit

,N,D,E
0,23,20.606306,0.006617
1,24,20.606306,0.006617
2,25,20.606306,0.006617
3,25,20.606306,0.006617
4,25,20.606306,0.095375
...,...,...,...
3995,25,20.482597,0.007658
3996,25,20.482597,0.015703
3997,25,20.482597,0.037362
3998,25,20.482597,0.146740


In [11]:
def my_model(theta,x):
    '''
    :param theta: Expected theta == [N, D, E]
    :param x: Expected list of scene arguments
    '''
    # Unpack model params
    N,D,E = theta
    # Model predictions
    y_pred = []
    # Get model predictins for each scene
    for scene in x:
        y_pred.append(abstraction_simulation_pp(scene_args[scene],int(N),D,E))
    # Convert to array
    y_pred = np.array(y_pred)
    return y_pred

In [12]:
# Observed RT
RT_y_mean = data.groupby('scene').rt.apply(np.mean)
RT_y_std = data.groupby('scene').rt.apply(np.std).mean()
RT_x = RT_y_mean.index.to_list()

In [13]:
N, D, E = model_fit.N.mean(), model_fit.D.mean(), model_fit.E.mean()

In [20]:
y_pred = my_model([N,D,E], RT_x)

/var/folders/6l/1kj5v3gx7r3264z3jy47_43c0000gn/T/ipykernel_16177/3302705581.py:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_pred = np.array(y_pred)


In [21]:
model = pd.DataFrame(y_pred, columns=["collision_m","samples","tick"])

In [23]:
model.tick = model.tick.apply(lambda x: x[0])

In [27]:
model['scene'] = RT_x

In [28]:
model

,collision_m,samples,tick,scene
0,0,1,1008,high_nocol_nosp_1
1,0,1,960,high_nocol_nosp_2
2,0,1,1008,high_nocol_nosp_3
3,0,1,1008,high_nocol_nosp_4
4,0,1,888,high_nocol_yessp_1
5,0,1,888,high_nocol_yessp_2
6,0,1,552,high_nocol_yessp_3
7,0,1,552,high_nocol_yessp_4
8,0,1,1008,high_yescol_nosp_1
9,0,1,1008,high_yescol_nosp_2
